## Introduction

**Craigslist is the world's largest collection of used vehicles for sale, yet it's very difficult to collect all of them in the same place. I built a scraper for a school project and expanded upon it later to create this dataset which includes every used vehicle entry within the United States on Craigslist.**

**In this project, we want to predict the used car price based on this dataset. This data is scraped every few months, it contains most all relevant information that Craigslist provides on car sales including columns like price, condition, manufacturer, latitude/longitude, and 18 other categories.**

In [109]:
import pandas as pd
import os


In [28]:
path = 'C:\\Users\\shoud\\Jupyter documents\\IBM ML professional certification\\Supervised machine learning regression\\Final project\\craigslistVehicles.csv'
data = pd.read_csv(path)
data.head(5)

,url,city,city_url,price,year,manufacturer,make,condition,cylinders,fuel,...,transmission,VIN,drive,size,type,paint_color,image_url,desc,lat,long
0,https://grandrapids.craigslist.org/cto/d/hasti...,"grand rapids, MI",https://grandrapids.craigslist.org,1500,2006.0,cadillac,cts,good,6 cylinders,gas,...,automatic,NaN,rwd,mid-size,coupe,blue,https://images.craigslist.org/00K0K_a9CZoZg2U8...,"2006 CtS Leather, Runs and drives Good.236k mil",42.643000,-85.293700
1,https://grandrapids.craigslist.org/cto/d/grand...,"grand rapids, MI",https://grandrapids.craigslist.org,8900,2009.0,lincoln,mkx,NaN,NaN,gas,...,automatic,NaN,NaN,NaN,NaN,NaN,https://images.craigslist.org/00a0a_9B4kPBDIWd...,"Selling our loaded 2009 Lincoln MKX with 119,0...",42.973700,-85.726500
2,https://grandrapids.craigslist.org/ctd/d/chesa...,"grand rapids, MI",https://grandrapids.craigslist.org,7995,2010.0,cadillac,srx premium collection,NaN,NaN,gas,...,automatic,3GYFNCEYXAS552363,NaN,NaN,NaN,NaN,https://images.craigslist.org/00X0X_8i0VRuk7Cv...,WE HAVE OVER 400 VEHICLES IN STOCK!\n\n View O...,43.186723,-84.163862
3,https://grandrapids.craigslist.org/ctd/d/chesa...,"grand rapids, MI",https://grandrapids.craigslist.org,6995,2007.0,NaN,hummer h3 4dr 4wd suv,NaN,NaN,gas,...,automatic,5GTDN13E478107380,NaN,NaN,NaN,NaN,https://images.craigslist.org/00b0b_ahkmUzr4cE...,WE HAVE OVER 400 VEHICLES IN STOCK!\n\n View O...,43.186723,-84.163862
4,https://grandrapids.craigslist.org/ctd/d/caled...,"grand rapids, MI",https://grandrapids.craigslist.org,20990,2010.0,ram,2500,excellent,6 cylinders,diesel,...,automatic,3D7UT2CL4AG113236,4wd,NaN,NaN,white,https://images.craigslist.org/00505_3DHY0kFrgb...,Great looking 2010 Ram 2500 ST w/6.7L 24V I6 4...,42.783714,-85.506777


## 1. Data preprocessing

In [29]:
drop_columns = ['url', 'city', 'city_url', 'make', 'title_status', 'VIN', 'size', 'image_url', 'desc', 'lat','long']
data = data.drop(columns = drop_columns)

In [30]:
data = data.dropna()

In [31]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144903 entries, 0 to 525836
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   price         144903 non-null  int64  
 1   year          144903 non-null  float64
 2   manufacturer  144903 non-null  object 
 3   condition     144903 non-null  object 
 4   cylinders     144903 non-null  object 
 5   fuel          144903 non-null  object 
 6   odometer      144903 non-null  float64
 7   transmission  144903 non-null  object 
 8   drive         144903 non-null  object 
 9   type          144903 non-null  object 
 10  paint_color   144903 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 13.3+ MB


In [32]:
data.head()

,price,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color
0,1500,2006.0,cadillac,good,6 cylinders,gas,236000.0,automatic,rwd,coupe,blue
5,4950,2010.0,subaru,good,4 cylinders,gas,253000.0,automatic,4wd,sedan,white
6,6850,2007.0,gmc,good,8 cylinders,gas,254000.0,automatic,4wd,wagon,black
7,7995,2007.0,lexus,excellent,6 cylinders,gas,146111.0,automatic,fwd,sedan,white
8,4995,2011.0,hyundai,excellent,4 cylinders,gas,115048.0,automatic,fwd,sedan,blue


In [33]:
data.columns

Index(['price', 'year', 'manufacturer', 'condition', 'cylinders', 'fuel',
       'odometer', 'transmission', 'drive', 'type', 'paint_color'],
      dtype='object')

In [34]:
categoric_col = [colname for colname in data.columns if data[colname].dtype == 'object']
numeric_col = [colname for colname in data.columns if data[colname].dtype == 'int64' or data[colname].dtype == 'float64']

In [35]:
data.describe()

,price,year,odometer
count,1.449030e+05,144903.000000,1.449030e+05
mean,6.198665e+04,2008.962865,1.134777e+05
std,1.120434e+07,7.835461,1.235806e+05
min,0.000000e+00,1900.000000,0.000000e+00
25%,4.000000e+03,2006.000000,6.491500e+04
50%,8.495000e+03,2010.000000,1.070000e+05
75%,1.599500e+04,2014.000000,1.499555e+05
max,3.009549e+09,2020.000000,1.000000e+07


In [36]:
df = data

In [13]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [39]:
df[categoric_col]

,manufacturer,condition,cylinders,fuel,transmission,drive,type,paint_color
0,6,2,5,2,0,2,3,1
5,36,2,3,2,0,0,9,10
6,14,2,6,2,0,0,12,0
7,23,0,5,2,0,1,9,10
8,17,0,3,2,0,1,9,1
...,...,...,...,...,...,...,...,...
525829,13,0,3,2,0,1,9,2
525830,37,2,3,2,0,2,10,0
525832,13,0,6,2,1,2,3,0
525835,30,2,3,2,0,0,0,10


In [38]:
for col in categoric_col:
    df[col] = le.fit_transform(df[col])

In [40]:
df.head(20)

,price,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color
0,1500,2006.0,6,2,5,2,236000.0,0,2,3,1
5,4950,2010.0,36,2,3,2,253000.0,0,0,9,10
6,6850,2007.0,14,2,6,2,254000.0,0,0,12,0
7,7995,2007.0,23,0,5,2,146111.0,0,1,9,10
8,4995,2011.0,17,0,3,2,115048.0,0,1,9,1
9,12995,2013.0,37,0,3,2,72936.0,0,1,4,10
12,8000,2010.0,3,2,5,2,150000.0,0,0,9,5
18,6995,2011.0,7,0,5,2,102000.0,0,1,9,1
21,39000,2017.0,7,0,6,2,43300.0,0,2,9,5
24,8900,2012.0,33,2,6,2,112000.0,0,2,8,10


In [42]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144903 entries, 0 to 525836
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   price         144903 non-null  int64  
 1   year          144903 non-null  float64
 2   manufacturer  144903 non-null  int32  
 3   condition     144903 non-null  int32  
 4   cylinders     144903 non-null  int32  
 5   fuel          144903 non-null  int32  
 6   odometer      144903 non-null  float64
 7   transmission  144903 non-null  int32  
 8   drive         144903 non-null  int32  
 9   type          144903 non-null  int32  
 10  paint_color   144903 non-null  int32  
dtypes: float64(2), int32(8), int64(1)
memory usage: 8.8 MB


In [43]:
df['year'] = (df['year']-1900).astype(int)

In [44]:
df['odometer'] = (df['odometer']).astype(int)

In [45]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 144903 entries, 0 to 525836
Data columns (total 11 columns):
 #   Column        Non-Null Count   Dtype
---  ------        --------------   -----
 0   price         144903 non-null  int64
 1   year          144903 non-null  int32
 2   manufacturer  144903 non-null  int32
 3   condition     144903 non-null  int32
 4   cylinders     144903 non-null  int32
 5   fuel          144903 non-null  int32
 6   odometer      144903 non-null  int32
 7   transmission  144903 non-null  int32
 8   drive         144903 non-null  int32
 9   type          144903 non-null  int32
 10  paint_color   144903 non-null  int32
dtypes: int32(10), int64(1)
memory usage: 7.7 MB


In [46]:
df.head(20)

,price,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color
0,1500,106,6,2,5,2,236000,0,2,3,1
5,4950,110,36,2,3,2,253000,0,0,9,10
6,6850,107,14,2,6,2,254000,0,0,12,0
7,7995,107,23,0,5,2,146111,0,1,9,10
8,4995,111,17,0,3,2,115048,0,1,9,1
9,12995,113,37,0,3,2,72936,0,1,4,10
12,8000,110,3,2,5,2,150000,0,0,9,5
18,6995,111,7,0,5,2,102000,0,1,9,1
21,39000,117,7,0,6,2,43300,0,2,9,5
24,8900,112,33,2,6,2,112000,0,2,8,10


## 2. EDA

In [49]:
df.describe()

,price,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color
count,1.449030e+05,144903.000000,144903.000000,144903.000000,144903.000000,144903.000000,1.449030e+05,144903.000000,144903.000000,144903.000000,144903.000000
mean,6.198665e+04,108.962865,18.252176,1.078190,4.658013,1.886310,1.134777e+05,0.117375,0.765139,6.047025,5.593915
std,1.120434e+07,7.835461,10.939406,1.163766,1.280567,0.535248,1.235806e+05,0.386070,0.774926,4.186399,3.993341
min,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000,0.000000,0.000000
25%,4.000000e+03,106.000000,8.000000,0.000000,3.000000,2.000000,6.491500e+04,0.000000,0.000000,0.000000,1.000000
50%,8.495000e+03,110.000000,14.000000,0.000000,5.000000,2.000000,1.070000e+05,0.000000,1.000000,8.000000,6.000000
75%,1.599500e+04,114.000000,30.000000,2.000000,6.000000,2.000000,1.499555e+05,0.000000,1.000000,9.000000,9.000000
max,3.009549e+09,120.000000,39.000000,5.000000,7.000000,4.000000,1.000000e+07,2.000000,2.000000,12.000000,11.000000


In [48]:
df['price'].value_counts()

0        6755
1        1743
4500     1603
3500     1593
2500     1528
         ... 
12761       1
5445        1
25470       1
26810       1
3297        1
Name: price, Length: 6578, dtype: int64

In [50]:
df = df[df['price'] > 1000]
df = df[df['price'] < 40000]
# Rounded ['odometer'] to 5000
df['odometer'] = df['odometer'] // 10000
df = df[df['year'] > 106]

In [51]:
df.describe()

,price,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color
count,91556.000000,91556.000000,91556.000000,91556.000000,91556.000000,91556.000000,91556.000000,91556.000000,91556.000000,91556.000000,91556.000000
mean,13999.936476,112.221799,18.420049,1.042706,4.521735,1.919044,9.575680,0.104188,0.709795,6.032603,5.543077
std,8625.365891,3.289994,10.938190,1.194800,1.260706,0.492375,9.578406,0.387579,0.731939,4.220463,4.045321
min,1025.000000,107.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,7000.000000,109.000000,10.000000,0.000000,3.000000,2.000000,5.000000,0.000000,0.000000,0.000000,1.000000
50%,11995.000000,112.000000,14.000000,0.000000,5.000000,2.000000,9.000000,0.000000,1.000000,8.000000,5.000000
75%,18995.000000,115.000000,30.000000,2.000000,6.000000,2.000000,13.000000,0.000000,1.000000,9.000000,9.000000
max,39999.000000,120.000000,39.000000,5.000000,7.000000,4.000000,963.000000,2.000000,2.000000,12.000000,11.000000


## 3. Prepare for model

In [66]:
target = df['price']
df = df.drop(['price'], axis=1)

In [69]:
from sklearn.preprocessing import StandardScaler 
scaler = StandardScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns = df.columns)

In [70]:
df.describe()

,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color
count,9.155600e+04,9.155600e+04,9.155600e+04,9.155600e+04,9.155600e+04,9.155600e+04,9.155600e+04,9.155600e+04,9.155600e+04,9.155600e+04
mean,-5.362341e-18,1.794944e-15,4.949434e-15,-4.592164e-15,-3.282591e-15,-2.589028e-16,2.557876e-14,-6.024147e-15,9.880738e-16,1.517206e-15
std,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00,1.000005e+00
min,-1.587184e+00,-1.684022e+00,-8.727084e-01,-3.586688e+00,-3.897548e+00,-9.997209e-01,-2.688180e-01,-9.697509e-01,-1.429378e+00,-1.370252e+00
25%,-9.792772e-01,-7.697887e-01,-8.727084e-01,-1.207056e+00,1.644202e-01,-4.777105e-01,-2.688180e-01,-9.697509e-01,-1.429378e+00,-1.123051e+00
50%,-6.741650e-02,-4.040955e-01,-8.727084e-01,3.793645e-01,1.644202e-01,-6.010223e-02,-2.688180e-01,3.964897e-01,4.661592e-01,-1.342490e-01
75%,8.444442e-01,1.058677e+00,8.012214e-01,1.172575e+00,1.644202e-01,3.575061e-01,-2.688180e-01,3.964897e-01,7.031014e-01,8.545530e-01
max,2.364212e+00,1.881487e+00,3.312116e+00,1.965785e+00,4.226388e+00,9.953948e+01,4.891450e+00,1.762730e+00,1.413928e+00,1.348954e+00


In [71]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df, target, test_size=0.3, random_state = 10010)

In [93]:
y_train

145956     4850
382183     4695
320817    12682
393757    39995
113164    13998
          ...  
47655     15887
261047    17695
397195    24997
117247    33000
485126    11450
Name: price, Length: 64089, dtype: int64

## 4. Model

In [85]:
from sklearn.linear_model import LinearRegression, SGDRegressor
from sklearn.svm import SVR, LinearSVR
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, ExtraTreesRegressor 
from sklearn.ensemble import BaggingRegressor, AdaBoostRegressor, VotingRegressor 
from sklearn.tree import DecisionTreeRegressor
import xgboost as xgb
import lightgbm as lgb

In [78]:
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error

In [94]:
def Regression_model(model,X_train,y_train):
    
    model = model.fit(X_train, y_train)
    y_predict = model.predict(X_test)  

    print('r2_square is: ', r2_score(y_test, y_predict))
    print('MAE is: ', mean_absolute_error(y_test, y_predict))
    print('MSE is: ', mean_squared_error(y_test, y_predict))

### 4.1 Linear regression

In [95]:
lr = LinearRegression()
Regression_model(lr, X_train, y_train)

r2_square is:  0.6357631815724045
MAE is:  3877.420434141576
MSE is:  26964067.80841615


### 4.2 SGDRegressor

In [96]:
SGDR = SGDRegressor()
Regression_model(SGDR, X_train, y_train)

r2_square is:  0.5953251232095702
MAE is:  4095.0295601521993
MSE is:  29957654.652391136


### 4.3 Support Vector Machine

In [97]:
svr = SVR()
Regression_model(svr, X_train, y_train)

r2_square is:  0.25806770461198003
MAE is:  5471.3784022896125
MSE is:  54924465.92428489


### 4.4 Random Forest

In [98]:
RFR = RandomForestRegressor()
Regression_model(RFR, X_train, y_train)

r2_square is:  0.8838415753347112
MAE is:  1806.4863369449017
MSE is:  8599085.76160666


### 4.5 GBR

In [99]:
GBR = GradientBoostingRegressor()
Regression_model(GBR, X_train, y_train)

r2_square is:  0.8103625167112394
MAE is:  2690.094223413921
MSE is:  14038663.033819543


### 4.6 ETR

In [100]:
ETR = ExtraTreesRegressor()
Regression_model(ETR, X_train, y_train)

r2_square is:  0.8784203922051684
MAE is:  1734.6374207063284
MSE is:  9000410.235441782


### 4.7 BR

In [102]:
BR = BaggingRegressor()
Regression_model(BR, X_train, y_train)

r2_square is:  0.8756316893490611
MAE is:  1877.9612747560175
MSE is:  9206854.968937464


### 4.8 ABR

In [103]:
GBR = AdaBoostRegressor()
Regression_model(GBR, X_train, y_train)

r2_square is:  0.6312422797981517
MAE is:  4286.999402401879
MSE is:  27298745.40230243


### 4.10 DTR

In [106]:
DTR = DecisionTreeRegressor()
Regression_model(GBR, X_train, y_train)

r2_square is:  0.6093398030219395
MAE is:  4436.142375683006
MSE is:  28920162.675590646


### 4.11 XGBR

In [107]:
XGBR = xgb.XGBRegressor()
Regression_model(XGBR, X_train, y_train)

r2_square is:  0.8711153487448511
MAE is:  2151.5699752113965
MSE is:  9541194.904212374


### 4.12 LGBR

In [108]:
LGBR = lgb.LGBMRegressor()
Regression_model(LGBR, X_train, y_train)

r2_square is:  0.85458355557768
MAE is:  2318.372493567771
MSE is:  10765026.129948074


## Summary

**In this project, we applied 12 different regression model to predict the used car price. Appearantly, support vector machine performs worst in this dataset. This is primarily because the typical svm can't draw a good hyperline to differentiate the data points. The Random Forest model performs best, for it can avoid the overfitting better than other emsemble methods. While the XGBR, LGBR, ETR, BR also have very high r2_score, more than 0.85.**

## Future work

**Although we have tried different regression model in this work, we didn't consider any tuning process. It is necessary to apply tuning method to get a better model with better performance metrics.**